<a href="https://colab.research.google.com/github/lucas-azdias/Conteudo-Universidade/blob/master/Ci%C3%AAncia%20da%20Computa%C3%A7%C3%A3o/6%C2%BA%20Per%C3%ADodo/Intelig%C3%AAncia%20Artificial/%5BTAREFAS%5D/TDE2%20-%20Busca%20Competitiva/TDE2%20Busca%20Competitiva.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**AUTORIA:**
- Guilherme Peres
- Henrique Schulz
- Lucas Dias
- Rafaela de Miranda
- Victor Portelinha

# **Ambiente**


## Bibliotecas

In [1]:
#Bibliotecas
import numpy as np


from numpy import inf
from os import name, system
from random import choice
from re import sub
from tabulate import tabulate
from time import time_ns
from typing import Any, Callable, Literal

## Constantes

### Constantes Utilitárias

In [2]:
# Constantes utilitárias
CMD_INPUT_MSG = " > "
CMD_CONTINUE_MSG = "" # "<Enter to continue>"

### Constantes Tabuleiro


In [3]:
# Constantes tabuleiro
BOARD_PLAYER_AGENTS_TYPE = Literal["HUMAN", "RANDOM", "MINIMAX", "MINIMAX ALPHA-BETA"]
BOARD_PLAYER_AGENTS = tuple(BOARD_PLAYER_AGENTS_TYPE.__args__)

## Funções


In [4]:
def is_intable(x:str) -> bool:
    # Verifies if the string is convertible to integer
    try:
        int(x)
        return True
    except:
        return False


def is_floatable(x:str) -> bool:
    # Verifies if the string is convertible to float
    try:
        float(x)
        return True
    except:
        return False


def calc_position(x: int, y:int, size:int) -> int:
    # Calculates the position in a 1D array based on 2D coordinates
    return x * size + y


def clear_screen() -> None:
    # Checks the operating system and run the clear screen command
    if name == 'nt':  # For Windows
        system('cls')
    else:  # For Unix/Linux/MacOS
        system('clear')


def input_info(msg:str=None, end="\n", verifier:Callable=lambda _: True) -> str:
    # Gets from the user any necessary information and rejects invalid ones
    while True:
        # Prints message if any
        if msg:
            print(msg)

        # Gets response from user and formats it
        response = sub(r'\s+', ' ', input(CMD_INPUT_MSG).strip())

        print(end=end)

        # If condition is valid, returns response
        if verifier(response):
            return response

## Classes

### Classe DUAL

In [5]:
class dual():
    # Class to emulate two values that must switch places at will

    def __init__(self, val1:Any, val2:Any) -> None:
        self.__val1__ = val1
        self.__val2__ = val2
        self.__initial__ = (val1, val2)


    def get_val1(self) -> Any:
        return self.__val1__


    def get_val2(self) -> Any:
        return self.__val2__


    def reset(self) -> None:
        self.__val1__, self.__val2__ = self.__initial__


    def change(self) -> None:
        self.__val1__, self.__val2__ = (self.__val2__, self.__val1__)

### Classe STATE_NODE

In [6]:
class state_node:

    def __init__(
            self,
            state:np.ndarray[np.int8]=None,
            utility:float=None,
            depth:int=None
        ) -> None:
        self.__state__ = state
        self.__utility__ = utility
        self.__depth__ = depth


    def __str__(self) -> str:
        return str(self.__state__) + (f" ({str(self.__utility__)})" if self.__utility__ else "")


    def get_state(self) -> np.ndarray[np.int8]:
        return self.__state__


    def get_utility(self) -> float:
        return self.__utility__


    def get_depth(self) -> int:
        return self.__depth__


    def set_utility(self, utility:float) -> None:
        self.__utility__ = utility

### Classe BOARD


In [7]:
class board:

    def __init__(
            self,
            size:int,
            refs:tuple[str]=("1", "-", "2"),
            utility_center:float=4,
            utility_corner:float=1
        ) -> None:
        # Constructor
        # Checks the minimum size for the board
        if not size >= 3:
            raise Exception("Size for the board must be equal or greater than 3")

        self.__size__ = size
        self.__round__ = 0
        self.__state__ = np.zeros(shape=(size * size), dtype=np.int8)

        # Initialize references to markers
        self.__player1_ref__ = refs[0]          # Player 1
        self.__empty_ref__ = refs[1]            # Empty
        self.__player2_ref__ = refs[2]          # Player 2

        # Map from references to markers
        self.__board_markers__ = {
            self.__player1_ref__: np.int8(-1),  # Player 1
            self.__empty_ref__: np.int8(0),     # Empty
            self.__player2_ref__: np.int8(1)    # Player 2
        }

        # Map from markers to references
        self.__board_refs__ = {
            marker: ref for ref, marker in self.__board_markers__.items()
        }

        # Utility weights for MINIMAX Algorithm
        self.__utility_center__ = utility_center
        self.__utility_corner__ = utility_corner

        # Initialize players' dual
        self.__players__ = dual(self.__player1_ref__, self.__player2_ref__)


    def __str__(self) -> str:
        # Returns a string representation of the board
        table = np.zeros(shape=(self.__size__, self.__size__), dtype=np.chararray)
        for i in range(self.__size__):
            for j in range(self.__size__):
                table[i][j] = self.__board_refs__[self.__state__[i * self.__size__ + j]]
        return tabulate(table, tablefmt="grid", stralign="center")


    def __next_player__(self) -> None:
        # Passes the player time if not in a terminal state (it guarantees that the player stays as winner)
        if not self.is_terminal_state()[0]:
            self.__players__.change()


    def __play__(self, pos:int, print_info:bool=False, delta_time_ns:int=None) -> None:
        # Executes a play if valid
        if not self.is_terminal_state()[0] and type(pos) == int and self.__state__[pos] == self.__board_markers__[self.__empty_ref__]:
            self.__round__ += 1
            self.__state__[pos] = self.__board_markers__[self.get_player()]

            # Print play info
            if print_info:
                print(f"Player {self.get_player()} executed play of round {self.__round__}", f" ({delta_time_ns / 1_000_000_000:.1f}s)" if delta_time_ns != None else "", sep="")
                # print(self)

            # Passes the player time
            self.__next_player__()

        else:
            raise Exception("Invalid play ignored")


    def __check_state__(self, callable:Callable, state:np.ndarray[np.int8]=None, dtype=np.int8) -> np.ndarray:
        # Applies a function through each line in the state of the board state
        if state is None:
            state = self.__state__

        plays = np.zeros(shape=(2 * self.__size__ + 2), dtype=dtype)

        # Vertical map
        for i in range(self.__size__):
            plays[i] = callable(state[(i * self.__size__):(i * self.__size__ + self.__size__)])

        # Horizontal map
        for i in range(self.__size__):
            plays[i + self.__size__] = callable(state[i::self.__size__])

        # Main diagonal map
        plays[2 * self.__size__] = callable(state[::(self.__size__ + 1)])

        # Secundary diagonal map
        plays[2 * self.__size__ + 1] = callable(state[(self.__size__ - 1)::(self.__size__ - 1)][:-1])

        return plays


    def __is_terminal_state__(self, state:np.ndarray[np.int8]) -> tuple[bool, str]:
        # Checks if the state is a goal state or a draw state and indicates the winner if any
        # For every check, sums all  elements
        line_results = self.__check_state__(lambda x: np.sum(x), state=state, dtype=np.int8)
        # Gets the index of the farthest element from 0
        max_index = np.argmax(np.abs(line_results))

        # Checks if it corresponds to the size (indicates that the check is full with one player's marker)
        is_goal = np.abs(line_results[max_index]) == self.__size__

        # If a line has been completed, returns the reference of the winner, else verifies if it is a draw
        if is_goal:
            is_draw = False
            winner = self.__board_refs__[line_results[max_index] / np.abs(line_results[max_index])]
        else:
            possible_wins = 0
            possible_wins += np.count_nonzero(self.__check_state__(lambda x: np.all(x >= 0), state=state, dtype=np.ndarray))
            possible_wins += np.count_nonzero(self.__check_state__(lambda x: np.all(x <= 0), state=state, dtype=np.ndarray))
            is_draw = possible_wins <= 0
            winner = None

        return (is_goal or is_draw, winner)


    def __evaluation__(self, state:np.ndarray[np.int8]) -> float:
        # Evaluation function for utility of states in MINIMAX Algorithm
        utility = 0
        max_utility = (self.__size__ * self.__size__) + 1
        player = self.__board_markers__[self.get_player()]
        depth = np.count_nonzero(state) - self.get_round()

        # Immediate Win/Loss
        is_terminal, winner = self.__is_terminal_state__(state)
        if is_terminal and winner and depth <= 2:
            if winner == self.get_player():
                utility += max_utility
            else:
                utility -= max_utility

        # Possible Wins/Losses
        player_callable = lambda x: np.all(x * player >= 0)
        adversary_player_callable = lambda x: np.all(x * player <= 0)
        utility += np.count_nonzero(self.__check_state__(player_callable, state=state, dtype=np.ndarray))
        utility -= np.count_nonzero(self.__check_state__(adversary_player_callable, state=state, dtype=np.ndarray))

        # Central position priority
        index = self.__size__ // 2
        if self.__size__ % 2 == 0: # Odd boards (1 central position)
            utility += state[calc_position(index    , index    , self.__size__)] * player * self.__utility_center__
        else: # Even boards (4 central positions)
            utility += state[calc_position(index - 1, index - 1, self.__size__)] * player * self.__utility_center__ / 4
            utility += state[calc_position(index - 1, index    , self.__size__)] * player * self.__utility_center__ / 4
            utility += state[calc_position(index    , index - 1, self.__size__)] * player * self.__utility_center__ / 4
            utility += state[calc_position(index    , index    , self.__size__)] * player * self.__utility_center__ / 4

        # Corners priority
        index = self.__size__ - 1
        utility += state[calc_position(0    , 0    , self.__size__)] * player * self.__utility_corner__
        utility += state[calc_position(0    , index, self.__size__)] * player * self.__utility_corner__
        utility += state[calc_position(index, 0    , self.__size__)] * player * self.__utility_corner__
        utility += state[calc_position(index, index, self.__size__)] * player * self.__utility_corner__

        # Depth penalty
        utility += max_utility - depth

        return utility


    def get_state(self) -> np.ndarray[np.int8]:
        # Returns the current state of the board
        return self.__state__.copy()


    def get_round(self) -> int:
        # Returns the current round number
        return self.__round__


    def get_player(self) -> str:
        # Returns the current player reference
        return self.__players__.get_val1()


    def get_adversary_player(self) -> str:
        # Returns the current adversary player reference
        return self.__players__.get_val2()


    def set_state(self, state:np.ndarray[np.int8]) -> None:
        # Set the state and resets all configs
        if not state.shape == np.shape(self.__size__ * self.__size__):
            raise ValueError("Invalid state passed")
        self.__round__ = 0
        self.__state__ = state
        self.__players__.reset()


    def is_terminal_state(self) -> tuple[bool, str]:
        # Checks if the current state is a goal state or a draw state and indicates the winner if any
        return self.__is_terminal_state__(self.__state__)


    def play(self, player_agent:BOARD_PLAYER_AGENTS_TYPE, depth_limit:int=np.inf, time_limit:float=np.inf, print_info:bool=False) -> None:
        # Raises exception if player agent is not a valid option
        if not player_agent in BOARD_PLAYER_AGENTS:
            raise ValueError(f"Invalid player agent passed")

        match player_agent:
            case "HUMAN":
                self.play_human(print_info)

            case "RANDOM":
                self.play_random(print_info)

            case "MINIMAX":
                self.play_minimax(depth_limit, time_limit, print_info)

            case "MINIMAX ALPHA-BETA":
                self.play_minimax_alphabeta(depth_limit, time_limit, print_info)


    def play_human(self, print_info:bool=False) -> None:
        # Raises exception if game is finished to prevent anymore plays
        if self.is_terminal_state()[0]:
            raise Exception("Terminal state reached")

        start_time_ns = time_ns()

        # Takes the row and the column as input from the player
        play = [None, None]
        is_valid_play = False
        while not is_valid_play:
            # Awaits for human input in format "0 0"
            print(f"<Player {self.get_player()} ({BOARD_PLAYER_AGENTS[0]})> Awaiting valid play:")
            play = input_info(end="").split(" ")

            try:
                # Verifies if play is convertable to integer
                if not is_intable(play[0]) or not is_intable(play[1]):
                    raise Exception("Non-convertable value to integer")

                # Converts the play row and column
                play[0] = int(play[0]) - 1
                play[1] = int(play[1]) - 1

                # Verifies if play is out of bounds
                if (not 0 <= play[0] < self.__size__) or (not 0 <= play[1] < self.__size__):
                    raise Exception("Out bounds position")

                # Calculates the position
                pos = calc_position(play[0], play[1], self.__size__)

                # Verifies if play is available
                if self.__state__[pos] != self.__board_markers__[self.__empty_ref__]:
                    raise Exception("Non-empty position")

            except Exception as error:
                # If the play is invalid, repeats
                print(str(error) + ". Try again!")
                continue

            # Confirms the play is valid
            is_valid_play = True

        # Does the play
        self.__play__(pos, print_info=print_info, delta_time_ns=(time_ns() - start_time_ns))


    def play_random(self, print_info:bool=False) -> None:
        # Raises exception if game is finished to prevent anymore plays
        if self.is_terminal_state()[0]:
            raise Exception("Terminal state reached")

        start_time_ns = time_ns()

        print(f"<Player {self.get_player()} ({BOARD_PLAYER_AGENTS[1]})> Playing...")

        # Filters all positions with an empty marker
        empty_pos = tuple(filter(lambda x: x[1] == self.__board_markers__[self.__empty_ref__], enumerate(self.__state__)))

        # Chooses randomly the index of an empty position
        pos = choice(empty_pos)[0]

        # Does the play
        self.__play__(pos, print_info=print_info, delta_time_ns=(time_ns() - start_time_ns))


    def play_minimax(self, depth_limit:int=np.inf, time_limit:float=np.inf, print_info:bool=False) -> None:
        # Raises exception if game is finished to prevent anymore plays
        if self.is_terminal_state()[0]:
            raise Exception("Terminal state reached")

        start_time_ns = time_ns()

        print(f"<Player {self.get_player()} ({BOARD_PLAYER_AGENTS[2]})> Playing...")

        # Converts time_limit from seconds to nanoseconds
        time_limit *= 1_000_000_000

        # MINIMAX Algorithm
        def minimax(path:list[state_node]) -> list[state_node]:
            # Emulates the agent MAX and the agent MIN
            def agent(path:list[state_node]) -> list[state_node]:
                # Chooses between agent MAX and agent MIN based on the depth
                if path[-1].get_depth() % 2 == 0:
                    player = self.get_player()
                    cur_path = [state_node(utility=-np.inf)]
                    agent_evaluator = max
                else:
                    player = self.get_adversary_player()
                    cur_path = [state_node(utility=np.inf)]
                    agent_evaluator = min

                # For each successor, pop it and takes MAX/MIN compared to the other successors
                for i in range(self.__size__ * self.__size__):
                    # Creates a sucessor for each empty space in the board state
                    if path[-1].get_state()[i] == self.__board_markers__[self.__empty_ref__]:
                        successor_state = path[-1].get_state().copy()
                        successor_state[i] = self.__board_markers__[player]
                        successor = state_node(state=successor_state, depth=(path[-1].get_depth() + 1))

                        cur_path = agent_evaluator(cur_path, minimax(path + [successor]), key=lambda x: x[-1].get_utility())

                return cur_path

            # If end of the tree (terminal nodes/nodes in goal state or if depth or time limit is reached),
            # calculates utility value, else runs agent MAX/MIN
            if self.__is_terminal_state__(path[-1].get_state())[0] or path[-1].get_depth() >= depth_limit or (time_ns() - start_time_ns) >= time_limit:
                utility = self.__evaluation__(path[-1].get_state())
                path[-1].set_utility(utility)
                return path
            else:
                return agent(path)

        # Executes MINIMAX Algorithm
        root = state_node(state=self.__state__.copy(), depth=0)
        if time_limit == np.inf:
            # Depth-limited execution (if depth limit is infinity, then searches the entire tree)
            path = minimax([root])
        else:
            # Iterative deepening execution (if shutdown in middle of execution, it still returns a decent path)
            max_depth_limit = depth_limit
            depth_limit = 1
            path = [root]
            while (depth_limit < max_depth_limit) and (not (time_ns() - start_time_ns) >= time_limit):
                depth_limit += 1
                old_path = path
                path = minimax([root])
            path = old_path # Gets the last complete search

        # Calculates the position based on the next state
        for i in range(self.__size__ * self.__size__):
            if self.__state__[i] != path[1].get_state()[i]:
                pos = i
                break

        # Does the play
        self.__play__(pos, print_info=print_info, delta_time_ns=(time_ns() - start_time_ns))


    def play_minimax_alphabeta(self, depth_limit:int=np.inf, time_limit:float=np.inf, print_info:bool=False) -> None:
        # Raises exception if game is finished to prevent anymore plays
        if self.is_terminal_state()[0]:
            raise Exception("Terminal state reached")

        start_time_ns = time_ns()

        print(f"<Player {self.get_player()} ({BOARD_PLAYER_AGENTS[3]})> Playing...")

        # Converts time_limit from seconds to nanoseconds
        time_limit *= 1_000_000_000

        # MINIMAX Algorithm with Alpha-Beta pruning
        def minimax_alphabeta(path:list[state_node], alpha:float=-np.inf, beta:float=np.inf) -> tuple[list[state_node], float, float]:
            # Emulates the agent MAX and the agent MIN
            def agent(path:list[state_node], alpha:float=-np.inf, beta:float=np.inf) -> tuple[list[state_node], float, float]:
                # Chooses between agent MAX and agent MIN based on the depth
                if path[-1].get_depth() % 2 == 0:
                    player = self.get_player()
                    cur_path = [state_node(utility=-np.inf)]
                    agent_evaluator = max
                else:
                    player = self.get_adversary_player()
                    cur_path = [state_node(utility=np.inf)]
                    agent_evaluator = min

                # For each successor, pop it and takes MAX/MIN compared to the other successors
                for i in range(self.__size__ * self.__size__):
                    # Creates a sucessor for each empty space in the board state
                    if path[-1].get_state()[i] == self.__board_markers__[self.__empty_ref__]:
                        # Alpha-Beta pruning
                        if alpha >= beta:
                            break

                        successor_state = path[-1].get_state().copy()
                        successor_state[i] = self.__board_markers__[player]
                        successor = state_node(state=successor_state, depth=(path[-1].get_depth() + 1))

                        new_path, new_alpha, new_beta = minimax_alphabeta(path + [successor], alpha, beta)
                        cur_path = agent_evaluator(cur_path, new_path, key=lambda x: x[-1].get_utility())

                        # Updates Alpha or Beta values
                        if path[-1].get_depth() % 2 == 0:
                            alpha = agent_evaluator(alpha, new_alpha, new_beta)
                        else:
                            beta = agent_evaluator(beta, new_alpha, new_beta)

                return cur_path, alpha, beta

            # If end of the tree (terminal nodes/nodes in goal state or if depth or time limit is reached),
            # calculates utility value, else runs agent MAX/MIN
            if self.__is_terminal_state__(path[-1].get_state())[0] or path[-1].get_depth() >= depth_limit or (time_ns() - start_time_ns) >= time_limit:
                utility = self.__evaluation__(path[-1].get_state())
                path[-1].set_utility(utility)
                return path, utility, utility
            else:
                return agent(path, alpha, beta)

        # Executes MINIMAX Algorithm with Alpha-Beta pruning
        root = state_node(state=self.__state__.copy(), depth=0)
        if time_limit == np.inf:
            # Depth-limited execution (if depth limit is infinity, then searches the entire tree)
            path = minimax_alphabeta([root])[0]
        else:
            # Iterative deepening execution (if shutdown in middle of execution, it still returns a decent path)
            max_depth_limit = depth_limit
            depth_limit = 1
            path = [root]
            while (depth_limit < max_depth_limit) and (not (time_ns() - start_time_ns) >= time_limit):
                depth_limit += 1
                old_path = path
                path = minimax_alphabeta([root])[0]
            path = old_path # Gets the last complete search

        # Calculates the position based on the next state
        for i in range(self.__size__ * self.__size__):
            if self.__state__[i] != path[1].get_state()[i]:
                pos = i
                break

        # Does the play
        self.__play__(pos, print_info=print_info, delta_time_ns=(time_ns() - start_time_ns))

# **TIC-TAC-TOE**



## Função principal

In [16]:
def main():
    # Create board in the specified size
    clear_screen()
    size = int(input_info(msg=f"Enter the size of the Tic-Tac-Toe board:", verifier=lambda x: is_intable(x) and int(x) >= 3))
    myboard = board(size)
    print( # input(
        f"Size {size} selected.\n" + CMD_CONTINUE_MSG
    )

    # Choose players based on the specified options
    clear_screen()
    menu = '\n'.join([f'[{i + 1}] {option}' for i, option in enumerate(BOARD_PLAYER_AGENTS)])
    print(f"{menu}\n")
    msg =  "Enter the playing option for the Player {}:"

    player_1 = int(input_info(msg=msg.format(myboard.get_player()), verifier=lambda x: is_intable(x)))
    player_2 = int(input_info(msg=msg.format(myboard.get_adversary_player()), verifier=lambda x: is_intable(x)))
    print( # input(
        f"Playing option {BOARD_PLAYER_AGENTS[player_1 - 1]} selected for Player {myboard.get_player()}.\n" +\
        f"Playing option {BOARD_PLAYER_AGENTS[player_2 - 1]} selected for Player {myboard.get_adversary_player()}.\n" +\
        CMD_CONTINUE_MSG
    )

    # If any MINIMAX or MINIMAX ALPHA=BETA player, gets depth limit and time limit
    def get_depth_time_limits(player:str) -> tuple[int, float]:
        clear_screen()
        depth_limit = input_info(msg=f"Enter the depth limit for the Player {player}:", verifier=lambda x: x == "" or is_intable(x))
        time_limit = input_info(msg=f"Enter the time limit for the Player {player}:", verifier=lambda x: x == "" or is_floatable(x))

        depth_limit_text = 'Depth limit ' + str(depth_limit) if depth_limit != "" else 'No depth limit'
        time_limit_text = 'Time limit ' + str(time_limit) + 's' if time_limit != "" else 'No time limit'
        print( # input(
            f"{depth_limit_text} selected for Player {player}.\n" +\
            f"{time_limit_text} selected for Player {player}.\n" +\
            CMD_CONTINUE_MSG
        )

        # If no depth limit or time limit, then infinity
        depth_limit = int(depth_limit) if depth_limit != "" else inf
        time_limit = float(time_limit) if time_limit != "" else inf

        return (depth_limit, time_limit)

    depth_limit_1, time_limit_1 = (None, None)
    depth_limit_2, time_limit_2 = (None, None)

    if player_1 in [3, 4]:
        clear_screen()
        depth_limit_1, time_limit_1 = get_depth_time_limits(myboard.get_player())

    if player_2 in [3, 4]:
        clear_screen()
        depth_limit_2, time_limit_2 = get_depth_time_limits(myboard.get_adversary_player())

    player_dual = dual(
        (BOARD_PLAYER_AGENTS[player_1 - 1], depth_limit_1, time_limit_1),
        (BOARD_PLAYER_AGENTS[player_2 - 1], depth_limit_2, time_limit_2)
    )

    # Game loop (breaks if whether the game was won or drawn)
    clear_screen()
    while True:
        # Terminal state verifier
        if myboard.is_terminal_state()[0]:
            if myboard.is_terminal_state()[1]:
                msg = f"The winner is Player {myboard.is_terminal_state()[1]}"
            else:
                msg = "It's a cat's game (draw)"
            break

        # Prints current round and board
        print(f"Round {myboard.get_round() + 1}")
        print(myboard)

        # Executes play of current player
        player, depth_limit, time_limit = player_dual.get_val1()
        myboard.play(player, depth_limit=depth_limit, time_limit=time_limit, print_info=True)

        # Switches player
        player_dual.change()

        print()

    # Prints final state with a message
    print(f"Final state\n{myboard}\n\n{msg}\n")

    print( # input(
        CMD_CONTINUE_MSG
    )

## Jogo

Para jogar, execute a função e siga os passos seguintes.

1. Selecionar tamanho do tabuleiro (Ex.: 3 indica um tabuleiro 3 × 3);
2. Selecionar quais serão as modalidades de jogadores;
3. Para cada jogador que seja do tipo MINIMAX ou MINIMAX ALPHA-BETA, indicar o limite de profundidade da árvore de busca e o limite de tempo para a busca (caso não queira limite, precionar ENTER);
4. Observar jogo;
5. Para cada que seja do tipo jogador HUMAN, realize a entrada da jogada na sua vez inserindo linha e coluna separados por espaço (Ex.: "1 2" indica uma jogada na posição de linha 1 e de coluna 2).

In [18]:
main()

Enter the size of the Tic-Tac-Toe board:
 > 4

Size 4 selected.


[1] HUMAN
[2] RANDOM
[3] MINIMAX
[4] MINIMAX ALPHA-BETA

Enter the playing option for the Player 1:
 > 1

Enter the playing option for the Player 2:
 > 4

Playing option HUMAN selected for Player 1.
Playing option MINIMAX ALPHA-BETA selected for Player 2.


Enter the depth limit for the Player 2:
 > 

Enter the time limit for the Player 2:
 > 1

No depth limit selected for Player 2.
Time limit 1s selected for Player 2.


Round 1
+---+---+---+---+
| - | - | - | - |
+---+---+---+---+
| - | - | - | - |
+---+---+---+---+
| - | - | - | - |
+---+---+---+---+
| - | - | - | - |
+---+---+---+---+
<Player 1 (HUMAN)> Awaiting valid play:
 > 2 2
Player 1 executed play of round 1 (3.1s)

Round 2
+---+---+---+---+
| - | - | - | - |
+---+---+---+---+
| - | 1 | - | - |
+---+---+---+---+
| - | - | - | - |
+---+---+---+---+
| - | - | - | - |
+---+---+---+---+
<Player 2 (MINIMAX ALPHA-BETA)> Playing...
Player 2 executed play of round 2 (1.